<a href="https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/00-comp-introducao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Tutoriais Iterativos em Jupyter para o Ensino de Compiladores

<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/dragon_cover.png" alt="Banner" width="40%"/>


Prof. Rogério Aparecido Gonçalves

e-mail: [rogerioag@utfpr.edu.br](mailto:rogerioag@utfpr.edu.br)

Universidade Tecnológica Federal do Paraná (UTFPR)

Departamento Acadêmico de Computação (DACOM-CM)

Curso de Bacharelado em Ciência da Computação.

Esse Tutorial foi desenvolvido para ser utilizado na disciplina de Compiladores dos cursos de Computação. O Tutorial de Compiladores faz parte do REA __COMP04: Criação de Tutoriais Iterativos e Testes para Compiladores__, desenvolvido no contexto do [EDITAL 37/2020 - PROGRAD](https://sei.utfpr.edu.br/sei/publicacoes/controlador_publicacoes.php?acao=publicacao_visualizar&id_documento=2039976&id_orgao_publicacao=0) para o Desenvolvimento de Recursos Educacionais Abertos.

## Resumo

_A disciplina de Compiladores é uma disciplina básica para a Computação. Nela os
alunos tem contato por meio do desenvolvimento de trabalhos com o projeto e desenvolvimento de uma ferramenta de compilação, um compilador. O projeto é dividido, normalmente, em quatro fases que correspondem às fases do processo de compilação, sendo elas Análise Léxica, Análise Sintática, Análise Semântica e Geração de Código. A disciplina de compiladores, pode ser considerada uma disciplina difícil em termos de conteúdo e das implementações dos projetos. O desenvolvimento da ferramenta de compilação utilizando a linguagem de programação Python, com bibliotecas como o PLY e llvmlite, facilita a implementação dos Analisadores Léxicos (lex) e Sintáticos (yacc) e a geração de código intermediário para LLVM, e possibilita que a infraestrutura e ferramentas
do LLVM possam ser utilizadas na geração do código final e executável. Com a popularização de notebooks iterativos em Jupyter, que permitem o aluno testar trechos de código, modificar e acrescentar novas funcionalidades e testar novamente. Na disciplina tem sido desenvolvidos tutoriais para cada uma das fases de compilação, no intuito de facilitar o desenvolvimento dos projetos, fornecendo um código inicial de referência para o desenvolvimento de cada fase._



## Introdução

O desenvolvimento de uma ferramenta de Compilação envolve pelo menos 4 fases básicas: _Análise Léxica_, _Análise Sintática_, _Análise Semântica_ e _Geração de Código_.

Para cada uma das fases temos alguma teoria vista em outras disciplinas do Curso de Ciência da Computação. Na _Análise Léxica_ para a identificação das _marcas_ ou _tokens_ utilizamos como base a teoria sobre _Autômatos_ e _Expressões Regulares (ER)_ vistas em _Linguagens Formais e Autômatos_ e no caso das _ERs_ utilizadas em diversas linguagens de programação e ferramentas do sistema.

A _Análise Sintática_ estrutura o código fonte de entrada como uma árvore sintática criada pela derivação das regras sintáticas da linguagem com a sequência de _tokens_ identificadas pelo Analisador Léxico. O resultado principal dessa fase é uma _Árvore Sintática_.

Já a _Análise Semântica_ de posse da Árvore gerada na fase anterior fará por meio de algoritmos de navegação ou caminhamento em árvores a verificação das regras semânticas da linguagem de trabalho. Verificadas todo o conjunto de regras semânticas definidos para a linguagem, temos que o programa estruturado na fase anterior está sintaticamente correto e também semanticamente correto, possui um significado válido. O resultado gerado por essa fase é uma árvore simplificada para a geração de Código.

A _Geração de Código_ é a fase final em alguns ciclos de desenvolvimento dependendo do tipo de código que será gerado, final (executável), assembly nativo ou código intermediário. No nosso tutorial a ideia é que a geração de código produza um código intermediário, uma representação intermediária, no nosso caso o código intermediário do LLVM, o LLVM-IR. A _Figura 1_ apresenta uma visão geral do processo de Compilação.

<!--![Visão Geral do Processo de Compilação](https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/cm-to-exe.png){ width=85% }-->

<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/cm-to-exe.png" alt="Visão Geral do Processo de Compilação" width="95%"/>

__Figura 1:__ Visão Geral do Processo de Compilação


## A Linguagem `C-`

A linguagem de Programação que iremos implementar como exemplo neste tutorial é a linguagem `C-` que é uma simplificação da linguagem `C-`, a escolha dessa linguagem foi pelo fato de grande parte das disciplinas introdutórias à programação, como Algoritmos dos cursos de Computação serem ministradas utilizando `C-`, assim teríamos a vantagem de ser uma linguagem conhecida pelos alunos. A versão simplificada `C-` é apresentada no \emph{Apêndice A} do livro texto utilizado na disciplina de Compiladores [1].

A sintaxe da linguagem \texttt{C-} tem as Regras Sintáticas apresentadas no _Código 1_.

```ebnf
program ::= declaration-list
declaration-list ::= declaration-list declaration | declaration
declaration ::= var-declaration | fun-declaration
var-declaration ::= type-specifier ID ; | type-specifier ID [ NUM ] ;
type-specifier ::= int | float | void
fun-declaration ::= type-specifier ID ( params ) compound-stmt
params ::= param-list | void
param-list ::= param-list , param | param
param ::= type-specifier ID | type-specifier ID [ ]
compound-stmt ::= { local-declarations statement-list }
local-declarations ::= local-declarations var-declaration | empty
statement-list ::= statement-list statement | empty
statement ::= expression-stmt | compound-stmt | selection-stmt | iteration-stmt | return-stmt
expression-stmt ::= expression ; | ;
selection-stmt ::= if ( expression ) statement | if ( expression ) statement else statement
iteration-stmt ::= while ( expression ) statement
return-stmt ::= return ; | return expression ;
expression ::= var = expression | simple-expression
var ::= ID | ID [ expression ]
simple-expression ::= additive-expression relop additive-expression | additive-expression
relop ::= <= | < | > | >= | == | !=
additive-expression ::= additive-expression addop term | term
addop ::= + | -
term ::= term mulop factor | factor
mulop ::= * | /
factor ::= ( expression ) | var | call | NUM
call ::= ID ( args )
args ::= arg-list | empty
arg-list ::= arg-list , expression | expression
```
__Código 1:__ Regras Sintáticas `C-`

## Diagramas Sintáticos

Diagramas Sintáticos são uma visualização gráfica das Regras Sintáticas. A _Figura 2_ apresenta a representação gráfica da regra para o _comando de seleção_ (IF). 

A regra para comando de seleção:

```selection-stmt ::= if '(' expression ')' statement ( else statement )?```

__selection-stmt:__

<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/selection-stmt.png" alt="Diagrama da Regra Sintática para Comando de Seleção" width="75%"/>

__Figura 2:__ Diagrama da Regra Sintática para Comando de Seleção


Todos os outros diagramas que representam as regras apresentadas no _Código 1_ estão disponíveis no repositório do _github_ do projeto [`rea-comp04-compiladores`](https://github.com/rogerioag/rea-comp04-compiladores/blob/main/cmmcompiler/ebnf/diagrams).

## Preparação do Ambiente

Como estamos utilizando ferramentas para gerar os _Analisadores Léxico e Sintático_ e na fase final gerarmos o código LLVM-IR na linguagem de programação _Python_, precisamos preparar o ambiente como algumas bibliotecas que serão utilizadas, dentre elas o [PLY](https://www.dabeaz.com/ply/ply.html) -- que gera a implementação da parte léxica e sintática, [AnyTree](https://anytree.readthedocs.io/en/latest) -- para a representação da árvore sintática, o [llvmlite](https://pypi.org/project/llvmlite/) -- para a geração do código LLVM-IR.


__Iremos instalar cada uma das ferramentas:__ [PLY](https://www.dabeaz.com/ply/ply.html), [AnyTree](https://anytree.readthedocs.io/en/latest), [graphviz](https://pypi.org/project/graphviz/) e [llvmlite](https://pypi.org/project/llvmlite/)


In [ ]:
!pip install ply

     |████████████████████████████████| 51kB 3.9MB/s 


In [ ]:
!pip install anytree

     |████████████████████████████████| 51kB 5.4MB/s 


In [ ]:
!pip install graphviz

In [ ]:
!pip install llvmlite

__Instalação de alguns _plugins_ para o Jupyter:__

In [ ]:
!jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip  --user
!jupyter nbextension enable highlighter/highlighter

Downloading: https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip -> /tmp/tmpRHcqzB/highlighter.zip
Extracting: /tmp/tmpRHcqzB/highlighter.zip -> /root/.local/share/jupyter/nbextensions
Enabling notebook extension highlighter/highlighter...
      - Validating: OK


In [ ]:
%%javascript
require("base/js/utils").load_extensions("highlighter/highlighter")

<IPython.core.display.Javascript object>

## Estrutura do Projeto do Código de Referência

Os arquivos utilizados nos _notebooks_ dos tutoriais estão disponíveis no `github`: [rea-comp04-compiladores](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler)

O código apresenta a seguinte estrutura:

<!--<img src="https://raw.githubusercontent.com/rogerioag/rea-comp04-compiladores/main/jupyter-notebooks/figuras/folder-black-code.svg"/>-->

| Diretório/Arquivo  | Conteúdo                                    |
| :------------ | :------------------------------------------ |
|  [cmmcompiler](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler) | Diretório raiz do Projeto do Compilador |
| [main.py](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/main.py) | Código principal |
| [lexer](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/lexer) | Módulo da Análise Léxica |
| [parser](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/parser) | Módulo da Análise Sintática |
| [semantic](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/semantic) | Módulo da Análise Semântica |
| [gencode](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/gencode) | Módulo da Geração de Código |
| [tests](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/tests) | Arquivos de Testes |
| [tree](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/tree) | Definição da Estrutura da Árvore |
| [utils](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/utils) | Definição de Funções auxiliares |
| [ebnf](https://github.com/rogerioag/rea-comp04-compiladores/tree/main/cmmcompiler/ebnf) | Documentos da especificação da linguagem `C-` |






## Fases do Desenvolvimento

Estruturamos o desenvolvimento em 4 fases, para cada uma delas preparamos um _notebook Jupyter_ com instruções e com algum código de _start_ para o desenvolvimento:

1. [Análise Léxica](https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/01-comp-analise-lexica-cmmlex.ipynb)

2. [Análise Sintática](https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/02-comp-analise-sintatica-cmmparser.ipynb)

3. [Análise Semântica](https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/03-comp-analise-semantica-cmmsemantic.ipynb)

4. [Geração de Código](https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/04-comp-geracao-de-codigo-cmmcodegen.ipynb)



## Referências

[1] LOUDEN, Kenneth C. **Compiladores:** princípios e práticas. São Paulo, SP: Thomson, c2004. xiv, 569 p. ISBN 8522104220.

[2] AHO, Alfred V. **Compiladores:** princípios, técnicas e ferramentas. 2. ed. São Paulo: Pearson Addison-Wesley, 2008. x, 634 p. ISBN 9788588639249.